In [7]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import math

Transformer Architecture for Text Classification

In [27]:
class MultiHeadAtention(nn.Module):
  def __init__(self, d_model, num_heads):
    super(MultiHeadAtention, self).__init__()
    assert d_model % num_heads == 0,  "d_models must be divisible by num_heads"

    self.d_model = d_model
    self.num_heads = num_heads
    self.d_k = d_model // num_heads

    # Linears layers for query, key, value and output

    self.W_q = nn.Linear(d_model, d_model)
    self.W_k = nn.Linear(d_model, d_model)
    self.W_v = nn.Linear(d_model, d_model)
    self.w_o = nn.Linear(d_model, d_model)

  def scale_dot_product(self, Q, K, V, mask=None):
    # Compute attention score
    attn_score = torch.matmul(Q, K.transpose(-2, -1)) / math.sqrt(self.d_k)

    # Apply mask if provide (useful for decoder transfer)
    if mask is not None:
      attn_score = attn_score.masked_fill(mask==0, float('-inf'))

    # Softmax probabilties of attention score
    attn_probs = F.softmax(attn_score, dim=-1)
    output = torch.matmul(attn_probs, V)

    return output

  def split_heads(self, x):
    batch_size, seq_length, d_model = x.size()
    return x.view(batch_size, seq_length, self.num_heads, self.d_k).transpose(1, 2)

  def combine_heads(self, x):
    batch_size,_, seq_length, _ = x.size()
    return x.transpose(1, 2).contiguous().view(batch_size, seq_length, self.d_model)

  def forward(self, Q, K, V, mask=None):

    # Linear projections
    K = self.split_heads(self.W_k(K))
    Q = self.split_heads(self.W_q(Q))
    V = self.split_heads(self.W_v(V))

    attn_output = self.scale_dot_product(Q, K, V, mask)
    # Combine heads and projection
    out = self.w_o(self.combine_heads(attn_output))
    return out

class PositionalEncoding(nn.Module):
  def __init__(self, d_model, max_seq_length=5000):
    super(PositionalEncoding, self).__init__()

    # Create positional Encoding
    pe = torch.zeros(max_seq_length, d_model)
    position = torch.arange(0, max_seq_length, dtype=torch.float).unsqueeze(1)
    div_term = torch.exp(torch.arange(0, d_model, 2).float() * -(math.log(10000.0) / d_model))

    pe[:, 0::2] = torch.sin(position * div_term)
    pe[:, 1::2] = torch.cos(position * div_term)

    self.register_buffer('pe', pe.unsqueeze(0))

  def forward(self, x):
    return x + self.pe[:, :x.size(1)]


class TransformerEncoderLayer(nn.Module):
  def __init__(self, d_model, num_heads, d_ff, dropout=0.1):
    super(TransformerEncoderLayer, self).__init__()

    self.self_attn = MultiHeadAtention(d_model, num_heads)
    self.feed_forward = nn.Sequential(
        nn.Linear(d_model, d_ff),
        nn.ReLU(),
        nn.Linear(d_ff, d_model)
    )

    self.norm1 = nn.LayerNorm(d_model)
    self.norm2 = nn.LayerNorm(d_model)

    self.dropout = nn.Dropout(dropout)

  def forward(self, x, mask=None):

    # self-attention on sub-layer
    attn_output = self.self_attn(x, x, x, mask)
    x = self.norm1(x + self.dropout(attn_output))

    #feed-forward sub-layer
    ff_output = self.feed_forward(x)
    x = self.norm2(x + self.dropout(ff_output))

    return x

class Transformer(nn.Module):
  def __init__(
      self,
      vocab_size,
      d_model=512,
      num_heads=8,
      num_layers=6,
      d_ff=2048,
      max_seq_length=5000,
      dropout=0.1,
      num_classes=10):

    super(Transformer, self).__init__()

    # Embedding_layer
    self.embedding = nn.Embedding(vocab_size, d_model)

    # Positional Encodding
    self.positional_encoding = PositionalEncoding(d_model, max_seq_length)

    # Encoder Layers

    self.encoder_layers = nn.ModuleList([
        TransformerEncoderLayer(d_model, num_heads, d_ff, dropout)
        for _ in range(num_layers)
    ])

    # Dropout and classifier layer
    self.dropout = nn.Dropout(dropout)
    self.classifier = nn.Linear(d_model, num_classes)

  def forward(self, x, mask=None):
    # Embedding and positional embedding
    x = self.embedding(x)
    x = self.positional_encoding(x)

    # Encoder stack

    for encoder_layer in self.encoder_layers:
      x = encoder_layer(x, mask)

    # Global Avg pooling

    x = x.mean(dim=1)

    x = self.classifier(self.dropout(x))

    return x

def create_transformers_model(vocab_size=10000, num_classes=10):
  model = Transformer(
      vocab_size=vocab_size,
      d_model=512,
      num_heads=8,
      num_layers=6,
      d_ff = 2048,
      num_classes=num_classes
  )

  return model

In [28]:
def demo():
  # Hyperparameter

  batch_size = 32
  seq_length = 100
  vocab_size = 10000
  num_classes = 10

  model = create_transformers_model(vocab_size=vocab_size, num_classes=num_classes)

  # create random input
  input = torch.randint(0, vocab_size, (batch_size, seq_length))

  output = model(input)

  print(f"Input shape: ", input.shape)
  print(f"Output shape: ", output.shape)

  return model



In [30]:
model = demo()

Input shape:  torch.Size([32, 100])
Output shape:  torch.Size([32, 10])


In [31]:
!pip install torchinfo --quiet

In [32]:
from torchinfo import summary
summary(model)

Layer (type:depth-idx)                   Param #
Transformer                              --
├─Embedding: 1-1                         5,120,000
├─PositionalEncoding: 1-2                --
├─ModuleList: 1-3                        --
│    └─TransformerEncoderLayer: 2-1      --
│    │    └─MultiHeadAtention: 3-1       1,050,624
│    │    └─Sequential: 3-2              2,099,712
│    │    └─LayerNorm: 3-3               1,024
│    │    └─LayerNorm: 3-4               1,024
│    │    └─Dropout: 3-5                 --
│    └─TransformerEncoderLayer: 2-2      --
│    │    └─MultiHeadAtention: 3-6       1,050,624
│    │    └─Sequential: 3-7              2,099,712
│    │    └─LayerNorm: 3-8               1,024
│    │    └─LayerNorm: 3-9               1,024
│    │    └─Dropout: 3-10                --
│    └─TransformerEncoderLayer: 2-3      --
│    │    └─MultiHeadAtention: 3-11      1,050,624
│    │    └─Sequential: 3-12             2,099,712
│    │    └─LayerNorm: 3-13              1,024
│    │ 

Transformer Architecture for Next word prediction

In [77]:
class MultiHeadAtention(nn.Module):
  def __init__(self, d_model, num_heads):
    super(MultiHeadAtention, self).__init__()
    assert d_model % num_heads == 0,  "d_models must be divisible by num_heads"

    self.d_model = d_model
    self.num_heads = num_heads
    self.d_k = d_model // num_heads

    # Linears layers for query, key, value and output

    self.W_q = nn.Linear(d_model, d_model)
    self.W_k = nn.Linear(d_model, d_model)
    self.W_v = nn.Linear(d_model, d_model)
    self.w_o = nn.Linear(d_model, d_model)

  def scale_dot_product(self, Q, K, V, mask=None):
    # Compute attention score
    attn_score = torch.matmul(Q, K.transpose(-2, -1)) / math.sqrt(self.d_k)

    # Apply mask if provide (useful for decoder transfer)
    if mask is not None:
      if mask.dim() == 2:
        mask = mask.unsqueeze(0).unsqueeze(0)
      elif mask.dim() == 3:
        mask = mask.unsqueeze(1)
      attn_score = attn_score.masked_fill(mask==0, float('-inf'))

    # Softmax probabilties of attention score
    attn_probs = F.softmax(attn_score, dim=-1)
    output = torch.matmul(attn_probs, V)

    return output

  def split_heads(self, x):
    batch_size, seq_length, d_model = x.size()
    return x.view(batch_size, seq_length, self.num_heads, self.d_k).transpose(1, 2)

  def combine_heads(self, x):
    batch_size,_, seq_length, _ = x.size()
    return x.transpose(1, 2).contiguous().view(batch_size, seq_length, self.d_model)

  def forward(self, Q, K, V, mask=None):

    # Linear projections
    K = self.split_heads(self.W_k(K))
    Q = self.split_heads(self.W_q(Q))
    V = self.split_heads(self.W_v(V))

    attn_output = self.scale_dot_product(Q, K, V, mask)
    # Combine heads and projection
    out = self.w_o(self.combine_heads(attn_output))
    return out

class PositionalEncoding(nn.Module):
  def __init__(self, d_model, max_seq_length=5000):
    super(PositionalEncoding, self).__init__()

    # Create positional Encoding
    pe = torch.zeros(max_seq_length, d_model)
    position = torch.arange(0, max_seq_length, dtype=torch.float).unsqueeze(1)
    div_term = torch.exp(torch.arange(0, d_model, 2).float() * -(math.log(10000.0) / d_model))

    pe[:, 0::2] = torch.sin(position * div_term)
    pe[:, 1::2] = torch.cos(position * div_term)

    self.register_buffer('pe', pe.unsqueeze(0))

  def forward(self, x):
    return x + self.pe[:, :x.size(1)]


class TransformerEncoderLayer(nn.Module):
  def __init__(self, d_model, num_heads, d_ff, dropout=0.1):
    super(TransformerEncoderLayer, self).__init__()

    self.self_attn = MultiHeadAtention(d_model, num_heads)
    self.feed_forward = nn.Sequential(
        nn.Linear(d_model, d_ff),
        nn.ReLU(),
        nn.Linear(d_ff, d_model)
    )

    self.norm1 = nn.LayerNorm(d_model)
    self.norm2 = nn.LayerNorm(d_model)

    self.dropout = nn.Dropout(dropout)

  def forward(self, x, mask=None):

    # self-attention on sub-layer
    attn_output = self.self_attn(x, x, x, mask)
    x = self.norm1(x + self.dropout(attn_output))

    #feed-forward sub-layer
    ff_output = self.feed_forward(x)
    x = self.norm2(x + self.dropout(ff_output))

    return x

class Transformer(nn.Module):
  def __init__(
      self,
      vocab_size,
      d_model=512,
      num_heads=8,
      num_layers=6,
      d_ff=2048,
      max_seq_length=5000,
      dropout=0.1):

    super(Transformer, self).__init__()

    # Embedding_layer
    self.embedding = nn.Embedding(vocab_size, d_model)

    # Positional Encodding
    self.positional_encoding = PositionalEncoding(d_model, max_seq_length)

    # Encoder Layers

    self.encoder_layers = nn.ModuleList([
        TransformerEncoderLayer(d_model, num_heads, d_ff, dropout)
        for _ in range(num_layers)
    ])

    # Dropout and classifier layer
    self.dropout = nn.Dropout(dropout)
    self.fc_out = nn.Linear(d_model, vocab_size)

    self.d_model = d_model
    self.max_seq_length = max_seq_length

  def generate_square_subsequent_mask(self, sz):
    """Generate a square mask to prevent attending future tokens."""
    mask = torch.triu(torch.ones(sz, sz), diagonal=1)
    return mask == 0

  def forward(self, x, mask=None):

    # create attention mask
    mask = self.generate_square_subsequent_mask(x.size(1)).to(x.device)

    # Embedding and positional embedding
    x = self.embedding(x) * math.sqrt(self.d_model)

    x = self.positional_encoding(x)

    # Encoder stack

    x = x.transpose(0, 1)

    for encoder_layer in self.encoder_layers:
      x = encoder_layer(x, mask)

    x = self.fc_out(self.dropout(x))

    return x

def create_transformers_model(vocab_size=10000):
  model = Transformer(
      vocab_size=vocab_size,
      d_model=512,
      num_heads=8,
      num_layers=6,
      d_ff = 2048
  )

  return model

In [78]:
model = create_transformers_model()

In [79]:
summary(model)

Layer (type:depth-idx)                   Param #
Transformer                              --
├─Embedding: 1-1                         5,120,000
├─PositionalEncoding: 1-2                --
├─ModuleList: 1-3                        --
│    └─TransformerEncoderLayer: 2-1      --
│    │    └─MultiHeadAtention: 3-1       1,050,624
│    │    └─Sequential: 3-2              2,099,712
│    │    └─LayerNorm: 3-3               1,024
│    │    └─LayerNorm: 3-4               1,024
│    │    └─Dropout: 3-5                 --
│    └─TransformerEncoderLayer: 2-2      --
│    │    └─MultiHeadAtention: 3-6       1,050,624
│    │    └─Sequential: 3-7              2,099,712
│    │    └─LayerNorm: 3-8               1,024
│    │    └─LayerNorm: 3-9               1,024
│    │    └─Dropout: 3-10                --
│    └─TransformerEncoderLayer: 2-3      --
│    │    └─MultiHeadAtention: 3-11      1,050,624
│    │    └─Sequential: 3-12             2,099,712
│    │    └─LayerNorm: 3-13              1,024
│    │ 

In [103]:
dummy_input = torch.randint(0, 10000, (16, 16))
print(dummy_input.shape)
torch.argmax(F.softmax(model(dummy_input), dim=2), dim=2).shape

torch.Size([16, 16])


torch.Size([16, 16])